# Feature Engineering

In [1]:
import pandas as pd

TO DO's:
- Limpar os campos de stockcode para conter apenas números
- Limpar os campos de stockcode para conter apenas 5 caracteres
- Identificar produtos que estão no description e substituir no stockcode*
- Limpar observações de valores negativos(Regra de negócio)
- Podemos continuar considerando todos os campos nulos além dos de StockCode e Quantity pois assim conseguimos entender a demanda irrestrita, ou seja, o que o produto de fato irá vender(Regra de Negócio)

In [2]:
df = pd.read_excel('../data/raw/online_retail_II.xlsx')

## Removendo campos nulos de stock, quantidade e data

In [3]:
df = df[
    (~df['StockCode'].isna()) & 
    (~df['Quantity'].isna()) & 
    (~df['InvoiceDate'].isna()) &
    (df['Quantity'] >= 0)
]

## Limpando campos de stockcode

In [4]:
df.loc[:, 'StockCode'] = df['StockCode'].astype('str')
df['stock_code_size'] = df['StockCode'].apply(lambda x: len(x))

# Foi feito estudo e identificado que valores acima de 8 caracteres não estão no padrão
df = df[(df['stock_code_size'] >= 5) & (df['stock_code_size'] < 8)]
df.loc[(df['stock_code_size'] > 5), 'StockCode'] = df['StockCode'].apply(
    lambda x: str(x)[:5])

# Removendo valores TEST  
df = df[df['StockCode'].str.isnumeric()]
df.loc[:, 'StockCode'] = df['StockCode'].astype('int')

## Agrupando no formato final

In [5]:
df_groupped = df.copy()

# Agrupando vendas mensais apenas
df_groupped['dt_year_month'] = (
    df_groupped['InvoiceDate'].dt.year*100
    + df_groupped['InvoiceDate'].dt.month
)
df_groupped['dt_year_month'] = pd.to_datetime(
    df_groupped['dt_year_month'], format='%Y%m')
df_groupped = df_groupped.groupby(
    ['StockCode', 'dt_year_month']).agg(
        qt_sales=('Quantity', 'sum'),
        qt_avg_price=('Price', 'mean')
).reset_index()
df_groupped.sort_values(['StockCode', 'dt_year_month'], inplace=True)

## Criando novas features

**Processo Estocástico:** Eventos observados indexados por um tempo
$\{X_t : t \in T\}$

Devemos criar features baseadas no tempo, sendo as mais comuns:
- Média móvel
- Lag
- Valores máximo e min em uma janela
- Frequência 
- Previsão de outros modelos
- Eventos específicos (datas comemorativas)

**Previsão séries temporais:**
- Devemos escolher qual o mês inicial de previsão para ser o label (lag 1)
- Não podemos utilizar os dados do mês atual para prevê-lo (vazamento de dados)

In [6]:
INITIAL_PREDICT_STR = 'qt_sales_lag_1'

### Lag

**OBS:** Idealmente é analisado a autocorrelação parcial de cada série para identificar quais são os melhores lags para se utilizar. Não iremos fazer isso apenas por se tratar de um modelo de machine learning (árvore de decisão)

In [7]:
number_of_lags = 6 

for lag in range(number_of_lags):
    df_groupped[f'qt_sales_lag_{lag+1}'] = df_groupped.groupby('StockCode')[
        'qt_sales'].shift(lag+1)
        
df_groupped

,StockCode,dt_year_month,qt_sales,qt_avg_price,qt_sales_lag_1,qt_sales_lag_2,qt_sales_lag_3,qt_sales_lag_4,qt_sales_lag_5,qt_sales_lag_6
0,10002,2009-12-01,219,1.336522,NaN,NaN,NaN,NaN,NaN,NaN
1,10002,2010-01-01,292,1.173889,219.0,NaN,NaN,NaN,NaN,NaN
2,10002,2010-02-01,257,0.958000,292.0,219.0,NaN,NaN,NaN,NaN
3,10002,2010-03-01,642,0.907895,257.0,292.0,219.0,NaN,NaN,NaN
4,10002,2010-04-01,1132,0.924167,642.0,257.0,292.0,219.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
29898,90214,2010-08-01,24,1.479091,15.0,247.0,21.0,31.0,68.0,20.0
29899,90214,2010-09-01,21,1.530000,24.0,15.0,247.0,21.0,31.0,68.0
29900,90214,2010-10-01,207,1.088710,21.0,24.0,15.0,247.0,21.0,31.0
29901,90214,2010-11-01,198,0.997945,207.0,21.0,24.0,15.0,247.0,21.0


### Média móvel

$MediaMovel = \frac{1}{k}\sum_{i=n-k+1}^n{p_i}$

, onde p são as observações, n é o número de valores da média móvel e k indica a posição inicial da média móvel

In [8]:
df_groupped['qt_sales_mavg_3'] = df_groupped.groupby('StockCode')[
    INITIAL_PREDICT_STR].transform(lambda x: x.rolling(3, 1).mean())
df_groupped['qt_sales_mavg_6'] = df_groupped.groupby('StockCode')[
    INITIAL_PREDICT_STR].transform(lambda x: x.rolling(6, 1).mean())

df_groupped

,StockCode,dt_year_month,qt_sales,qt_avg_price,qt_sales_lag_1,qt_sales_lag_2,qt_sales_lag_3,qt_sales_lag_4,qt_sales_lag_5,qt_sales_lag_6,qt_sales_mavg_3,qt_sales_mavg_6
0,10002,2009-12-01,219,1.336522,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10002,2010-01-01,292,1.173889,219.0,NaN,NaN,NaN,NaN,NaN,219.000000,219.000000
2,10002,2010-02-01,257,0.958000,292.0,219.0,NaN,NaN,NaN,NaN,255.500000,255.500000
3,10002,2010-03-01,642,0.907895,257.0,292.0,219.0,NaN,NaN,NaN,256.000000,256.000000
4,10002,2010-04-01,1132,0.924167,642.0,257.0,292.0,219.0,NaN,NaN,397.000000,352.500000
...,...,...,...,...,...,...,...,...,...,...,...,...
29898,90214,2010-08-01,24,1.479091,15.0,247.0,21.0,31.0,68.0,20.0,94.333333,67.000000
29899,90214,2010-09-01,21,1.530000,24.0,15.0,247.0,21.0,31.0,68.0,95.333333,67.666667
29900,90214,2010-10-01,207,1.088710,21.0,24.0,15.0,247.0,21.0,31.0,20.000000,59.833333
29901,90214,2010-11-01,198,0.997945,207.0,21.0,24.0,15.0,247.0,21.0,84.000000,89.166667


In [9]:
df_groupped.fillna(0, inplace=True)
df_groupped.to_parquet('../data/temp/df_model.parquet')